.[머신러닝 학습단계(7단계)].
1. 문제를 정의하는 단계 : 어떤 문제를 어떻게 해결할 것인지 정의
2. 데이터 수집 : 1단계에서 정의된 문제를 해결하기 위해 데이터를 수집하는 단계
    - 변수 = pd.DataFrame('가지고 온 데이터')
3. 데이터 전처리 : 결측치와 이상치를 수정하는단계, 결측치, 이상치를 확인하고 수정
    - 변수.info() : 결측치 확인
    - 변수.describe() : 이상치 확인
 4. 탐색적 데이터 분석(EDA) : 시간이 충분치 않을 때는 많이 생략되는 과정, 데이터를 그래프로 시각화, 기술통계값으로 데이터를 살펴보는 과정
5. 모델 선택 및 하이퍼파라미터 튜닝 : 
    - 사용할 모델을 불러와서 파라미터를 수정 
      트리모델 = DecisionTreeClassifier(
    - 데이터 분리(문제/정답-인덱서, 훈련/평가-train_test_split())
       X : 문제(데이터프레임 형태), y : 정답(시리즈 형태)
       X_train, X-test, y_train, y_test = train_test_split(X,y, test_size=0.3)
6. 학습 : 모델이 규칙을 찾기위한 과정, 훈련데이터(train data)로 학습
    - 트리모델.fit(X_train, y_train)
7. 평가 : 모델 자체 평가, 훈련/평가용 데이터의 점수를 확인 => 과대적합/과소적합 여부 확인
    - 교차 검증 X : 트리모델.score(X-train, y-train)
                    트리모델.score(X_test, y_test)
    - 교차 검증 O : cross_val_score(X-train, y-train)
                    cross_val_score(X_test, y_test)
   모델이 예측하는 값 확인 
   - 트리모델.predict([[이차원 형태의 새로운 데이터]]

### 문제 정의

### 데이터 수집
 - kaggle 사이트에서 데이터 다운로드

In [1]:
# train과 test 데이터 가져오기
# train 변수 : train.csv 파일
# test 변수 : test.csv 파일
import pandas as pd 
train = pd.read_csv('./Data/titanic/train.csv')
test = pd.read_csv('./Data/titanic/test.csv')


print(train.shape)
print(test.shape)
# train과 test의 특성(컬럼)의 차이는 Survived 컬럼의 차이
# 머신러닝 학습을 통해서 찾은 규칙을 기반으로 예측해야 하는 컬럼.


(891, 12)
(418, 11)


#### 컬럼들의 특징
![특징]("/titanic/info.png")

### 데이터 전처리


#### 이상치 확인

In [2]:
train.info()
# 결측치 컬럼 : Age, Cabin, Embarked
# 머신러닝 학습을 위해 숫자형 자료로 변경 컬럼 : Name, Sex,Ticket,Cabin,Embarked


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [3]:
test.info()
# 결측치 컬럼 : Age, fair, Cabin : RangeIndex: 418 개가 아닌 값들
# 머신러닝 학습을 위해 숫자형 자료로 변경 컬럼 :Name
# Name, sex, Ticket, cabin, Embarked => encoding 필요

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


#### 이상치 확인
 - describe()

In [4]:
train.describe()
# passengerld : 승객의 번호를 의미, 필요없는 데이터 -> 삭제
# Survived(0,1) -> 38%가 생존
# Pclass : 객실의 등급(1,2,3)으로 평규읜 의미 (std)가 없음
# Age : 평균값이 의미가 있음. 20~30대 승객이 많음.
# SibSp : 형제 자매/ 배우자 수 --> 이상치 의미가 없음
# Parch : 부모/자식 수  --> 이상치 의미가 없음
# fare : 평균은 어느정도 의미가 있음.
#  평균과 중앙값을 비교했을 때 중앙값에 비해 평균이 높다는 의미는?
#  1.데이터 수집이 잘못
#  2.티켓의 가격이 잘못
#  3.진짜로 1등급을 넘어서는 가격이 존재 

# Fare 컬럼을 제외하고는 특별한 이상치가 있다고 볼 수 없음.

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


#### Embarked 결측치 채우기
- 탑승항구 : S, C, Q 정보
- 결측치 수가 상대적으로 작기때문에(2) 먼저 처리

In [5]:
# 해당 컬럼의 데이터 갯수를 세어주는 함수 : value_counts()
train['Embarked'].value_counts()

# 결측치를 채울 때 주의할 점
# - 어떤 기준으로 값을 채울 것인가 확인이 중요
# 탑승하는 항구의 값을 채우기 위해 다른 컬럼이나 추가적인 정보를 확인  필요. 
# 탑승하는 항구별로 티켓의 가격이 다른지 확인하는 것도 의미가 있을 수 있음 .

# 결측치가 2개이므로 전체적인 흐름에 영향을 주지 않을 것 같아서 ==> 대표값으로 측정

S    644
C    168
Q     77
Name: Embarked, dtype: int64

##### train데이터 - 결측치 채우기

In [6]:
# 결측치 2개(891-> 889)를 최빈값(S) ==> 대표값으로 채우기 
#  전체 결측치를 한번에 채워주는 함수 : fillna(입력값)

train['Embarked'].fillna('S', inplace=True) #inplace = True : 변경된 값을 저장

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


##### test데이터 -Fare 결측치 채우기 

In [8]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [9]:
#test['Fare'].value_counts()
# Embarked 컬럼처럼 그 컬럼의 최빈값으로 채우는 게 의미가 없음
# 연속적인 숫자값으로 가장 최빈값이 7.75 ==> 평균도 아니고 중앙값에도 미치지 못하는 값

# Fare 컬럼의 중앙값으로 사용하여 결측치를 채워보자
test['Fare'].fillna(14.454200,inplace=True) # 중앙값으로 변경허고 저장

In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         418 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


#### Age 결측치 채우기
# - 다른른 컴럼들과의 상관관계를 사용해서 결측치를 채우기

In [11]:
# 수치형 컬럼들 간의 상관관계를 나타내는 함수 : corr()
# 그래프의 값이 마이너스에 가까울때는 반비례 관계. 1에 가까울 때는 비례 관계.
# 나이와 Pclass,sibSp,Parch는 반비례관계, Fard는 비례관계
# -1 ~ 1 사이의 숫자로 표현 
# -1에 가까울 수록 반비례 관계 : 나이가 많을 수록 객실등급은 1등급에 가까울 거다.
# 1에 가까울 수록 (정)비례 : 
train.corr()

# corr 함수가 오류가 날 경우 train.select_dtypes(include=['int64', 'float64']).corr()
# 수치 앞의 부호는 정비례. 반비례인지 판단 기준
# 부호뒤의 숫자의 절대값이 1에 가까울 수록 서로 영향을 준다고 판단
# 형제자매/배우자수, 객실등급이 나이와 상관이 있을 것이다.

C:\Users\gjaischool\AppData\Local\Temp\ipykernel_17028\2458440882.py:7: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  train.corr()


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


In [12]:
# Age 컬럼을 Pclass컬럼과 비교해서 결측치 채우기

train[['Pclass','Age']] # -> pclass 별로 묶어서 중앙값 확인

train[['Pclass', 'Age']].groupby(by='Pclass').median()
# Pclass 별로 탑승한 승객의 나이의 중앙값

#Pclass = 1,2,3 : 숫자자료이지만 등급을 나타내는 범주형 자료 

,Age
Pclass,
1,37.0
2,29.0
3,24.0


In [13]:
# Age 컬럼을 Pclass 먼저, Sex 컬럼 추가하기 
# Pclass 컬럼별로 탑승한 승객의 나이의 중앙값

# 3개의 컬럼을 보기 위해서 age_table로 저장 (테이블 형태로)
age_table= train[['Pclass','Sex','Age']].groupby(by=['Pclass','Sex']).median()
age_table

Age
Pclass Sex         
1      female  35.0
       male    40.0
2      female  28.0
       male    30.0
3      female  21.5
       male    25.0

In [14]:
train.iloc[2]

PassengerId                         3
Survived                            1
Pclass                              3
Name           Heikkinen, Miss. Laina
Sex                            female
Age                              26.0
SibSp                               0
Parch                               0
Ticket               STON/O2. 3101282
Fare                            7.925
Cabin                             NaN
Embarked                            S
Name: 2, dtype: object

In [15]:
# 1. 함수 정의 
# 하나의 행을 가져와서
# Age가 결측치가 아닌 경우 --> 원래 가지고 있던 값을 그대로 사용
# Age가 결측치라면 --> Pclass와 Sex 컬럼을 확인하고 해당하는 값을 age_table에서 검색해서 해당 값으로 채우기 
# 꺼내오는 하나의 행 정보 : 넘파이 배열형태 
import numpy as np

# 함수를 만들 때 주의할 점 
# 행단위로 데이터가 들어올 것으로 예상하고 만들기 
def fill_age(person) : 
    if np.isnan(person['Age']) : # 꺼내온 정보에서 Age 컬럼이 결측치 0 
        return age_table.loc[person['Pclass'], person['Sex']]['Age']
        # 결측치라면 age_table의 객실등급과 성별을 확인하고 age 값을 적음
    else : # Age 컬럼이 결측치가 아님
        return person['Age'] #가지고 있는 값 사용 

In [17]:
# 2. 사용자 승객의 정보를 확인하고 함수 적용 ==> apply()
# apply() : 행과 열단위의 복잡한 계산을 할 때 사용하는 함수 
age_table.loc[3, 'male']['Age']

25.0

### 탐색적 데이터 분석 (EDA)

### 모델 선택 및 하이퍼 파라미터 튜닝

### 학습

### 평가 